In [ ]:
import sys; sys.path.append('..')
import numpy as np, elastic_rods
from bending_validation import suppress_stdout
from linkage_vis import LinkageViewer
from open_linkage import open_linkage

l = elastic_rods.RodLinkage('../examples/optimized/data/AsymmWingsPointy/1935b524-e979-4340-9245-326f69b6eae0.obj')
mat = elastic_rods.RodMaterial('Rectangle', 20000, 0.3, [12,8], stiffAxis=elastic_rods.StiffAxis.D1)
l.setMaterial(mat)

l.setPerSegmentRestLength(np.loadtxt('../examples/optimized/data/AsymmWingsPointy/design_parameters.txt'))

driver = l.centralJoint()
jdo = l.dofOffsetForJoint(driver)

fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
with suppress_stdout(): elastic_rods.compute_equilibrium(l, fixedVars=fixedVars)
view = LinkageViewer(l, width=1024)
view.setCameraParams(((0.4075185454716348, 3.1892642201046715, 0.3099480992441177),
 (0.15364528336486324, 0.2839547329660347, 0.9464474821805594),
 (0.0, 0.0, 0.0)))
view.show()

In [ ]:
def equilibriumSolver(tgtAngle, l, opts, fv):
    opts.beta = 1e-8
    opts.gradTol = 1e-4
    opts.niter = 100
    opts.useIdentityMetric = False
    return elastic_rods.compute_equilibrium(l, tgtAngle, options=opts, fixedVars=fv)

In [ ]:
open_linkage(l, driver, np.deg2rad(113) - l.averageJointAngle, 50, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=10, useTargetAngleConstraint=True);
# open_linkage(l, driver, np.deg2rad(183) - l.averageJointAngle, 50, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=10, useTargetAngleConstraint=False);

In [ ]:
mkdir average_angle_deploy

In [ ]:
from matplotlib import pyplot as plt
angles = np.linspace(0.3750346, np.deg2rad(113), 240)
for i, angle in enumerate(angles):
    with suppress_stdout(): open_linkage(l, driver, angle - l.averageJointAngle, 1, None, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=10, useTargetAngleConstraint=True);
    l.saveVisualizationGeometry(f'average_angle_deploy/frame_{i}.msh', averagedMaterialFrames=True)
    
    jointAngles = np.rad2deg(l.getDoFs()[l.jointAngleDoFIndices()])
    plt.figure(figsize=(9,6))
    plt.hist(jointAngles, range=[0, 180], bins=50)
    plt.ylim(0, 65)
    plt.gca().axvline(linewidth=4, x=np.rad2deg(angle), color='black')
    plt.tight_layout()
    plt.savefig(f'average_angle_deploy/hist_{i}.png')
    plt.close()

In [ ]:
jointAngles = np.rad2deg(l.getDoFs()[l.jointAngleDoFIndices()])
open_linkage(l, driver, np.deg2rad(113) - l.averageJointAngle, 50, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=10, useTargetAngleConstraint=True);

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(9,6))
plt.hist(jointAngles, range=[0, 180], bins=50)
plt.ylim(0, 65)
plt.gca().axvline(linewidth=4, x=100, color='black')
plt.tight_layout()